# Create a Simple Reflex-Based Lunar Lander Agent

In this example, we will use Gymnasium, an environment to train agents via reinforcement learning (RL). We will not use RL here but just use the environment with a custom simple reflex-based agent.

## Install Gymnasium

The documentation for Gymnasium is available at https://gymnasium.farama.org/

Steps:
1. Create a new folder and open it with VS Code and install all needed Python Extensions in VS Code.
2. Create a new virtual environment (CTRL-Shift P Python Create Environment...)
3. I needed to install swig and the Python C++ headers on WSL2 via the terminal
    * `sudo apt install swig`
    * `sudo apt-get install python3-dev`
4. Install gymnasium with the needed extras

In [ ]:
%pip install -q swig
%pip install -q gymnasium[box2d,classic_control]

Note: you may need to restart the kernel to use updated packages.


## The Lunar Lander Environment

The documentation of the environment is available at: https://gymnasium.farama.org/environments/box2d/lunar_lander/

* Performance Measure: A reward of -100 or +100 points for crashing or landing safely respectively. We do not use
  intermediate rewards here.

* Environment: This environment is a classic rocket trajectory optimization problem. A ship needs to land safely. The space is **continuous** with
  x and y coordinates in the range [-2.5, 2.5]. The landing pad is at coordinate (0,0).

* Actuators:  According to Pontryagin’s
  maximum principle, it is optimal to fire the engine at full throttle or turn it off. This is the reason why this environment has discrete actions: engine on or off. There are four discrete actions available:

    - 0: do nothing
    - 1: fire left orientation engine
    - 2: fire main engine
    - 3: fire right orientation engine

* Sensors: Each observation is an 8-dimensional vector: the coordinates of the lander in x & y, its linear velocities in x & y, its angle, its angular velocity, and two booleans that represent whether each leg is in contact with the ground or not.

Gymnasim environments are implemented as classes with a `make` method to create the environment, a `reset` method, and a `step` method to execute an action.
To use it with an agent function that expects percetps and returns an action, we need write glue code that connects the environment with the agent function.

In [2]:
import gymnasium as gym

def run_episode(agent_function, max_steps=1000):
    """Run one episode in the LunarLander-v3 environment using the provided agent."""

    # Initialize the environment
    env = gym.make("LunarLander-v3", render_mode="human")

    # Reset the environment to generate the first observation (use seed=42 in reset to get reproducible results)
    observation, info = env.reset()

    # run one episode
    for _ in range(max_steps):
        # call the agent function to select an action
        action = agent_function(observation)

        print (f"Obs: {observation} -> Action: {action}")

        # step: execute an action in the environment
        observation, reward, terminated, truncated, info = env.step(action)

        env.render()

        if terminated:
            print(f"Final Reward: {reward}")
            break

    env.close()
    return reward

Note: `env.render()` shows the environment when the notebook is locally run (e.g., in VScode). On Colab, you cannot see the environment because the code is run on a headless server (i.e., a server without a display). There are some workarounds you can google.

## Example: A Random Agent

We ranomly return one of the actions. The environment accepts the integers 0-3.


In [ ]:
# import numpy as np

# def random_agent_function(observation):
#     """A random agent that selects actions uniformly at random. It ignores the observation."""
#     return np.random.choice([0, 1, 2, 3], p=[0.25, 0.25, 0.25, 0.25])

# run_episode(random_agent_function)

Obs: [ 0.00604944  1.4038899   0.61273247 -0.31246874 -0.00700306 -0.13879311
  0.          0.        ] -> Action: 0
Obs: [ 0.01209917  1.3962835   0.6119045  -0.3381167  -0.01386151 -0.1371814
  0.          0.        ] -> Action: 1
Obs: [ 0.01808014  1.3880674   0.60327613 -0.3652163  -0.01898345 -0.10244817
  0.          0.        ] -> Action: 1
Obs: [ 0.02398119  1.3792539   0.59325033 -0.39175427 -0.02209131 -0.06216275
  0.          0.        ] -> Action: 3
Obs: [ 0.02996778  1.3698475   0.6039585  -0.41813973 -0.02734182 -0.10501935
  0.          0.        ] -> Action: 1
Obs: [ 0.03587885  1.3598377   0.5944906  -0.44495237 -0.03069059 -0.06698141
  0.          0.        ] -> Action: 3
Obs: [ 0.04185562  1.3492153   0.6027326  -0.47221822 -0.03569708 -0.10013922
  0.          0.        ] -> Action: 0
Obs: [ 0.04783259  1.3379933   0.60274845 -0.49888843 -0.04070169 -0.10010101
  0.          0.        ] -> Action: 1
Obs: [ 0.053722    1.326173    0.59178454 -0.52542114 -0.04350451

-100

## A Simple Reflex-Based Agent

To make the code easier to read, we use enumerations for actions (integers) and observations (index in the observation vector).

In [3]:
from enum import Enum

class Act(Enum):
    LEFT = 1
    RIGHT = 3
    MAIN = 2
    NO_OP = 0

class Obs(Enum):
    X = 0
    Y = 1
    VX = 2
    VY = 3
    ANGLE = 4
    ANGULAR_VELOCITY = 5
    LEFT_LEG_CONTACT = 6
    RIGHT_LEG_CONTACT = 7


In [12]:
def rocket_agent_function(observation):
    """A simple agent function."""

    # run the main thruster, if the lander is falling too fast
    if observation[Obs.VY.value] < -.3:
        return Act.MAIN.value

    return Act.NO_OP.value

run_episode(rocket_agent_function)

Obs: [-0.00742388  1.4060056  -0.7519841  -0.21845403  0.00860932  0.17033552
  0.          0.        ] -> Action: 0
Obs: [-0.01484842  1.4005146  -0.75096804 -0.24411255  0.01702622  0.1683543
  0.          0.        ] -> Action: 0
Obs: [-0.02227325  1.3944244  -0.75099397 -0.27079618  0.02543812  0.16825426
  0.          0.        ] -> Action: 0
Obs: [-0.02969837  1.3877351  -0.7510185  -0.29747322  0.03384895  0.16823246
  0.          0.        ] -> Action: 0
Obs: [-0.03712378  1.3804467  -0.751043   -0.3241472   0.04225845  0.16820545
  0.          0.        ] -> Action: 2
Obs: [-0.0444005   1.3734202  -0.7370015  -0.31257683  0.0514937   0.18472221
  0.          0.        ] -> Action: 2
Obs: [-0.05186386  1.3663275  -0.75485384 -0.31554064  0.0599256   0.16865356
  0.          0.        ] -> Action: 2
Obs: [-0.05952968  1.3596914  -0.7743226  -0.29527262  0.06759591  0.15342054
  0.          0.        ] -> Action: 0
Obs: [-0.0671957   1.3524557  -0.77434385 -0.32195243  0.07526534

-100

## Evaluating the Agent

Run the agent on 100 problems and report the average reward.

In [13]:
import numpy as np

def run_episode_test(agent_function):
    """Run one episode in the LunarLander-v3 environment using the provided agent."""

    # Initialise the environment
    env = gym.make("LunarLander-v3", render_mode=None)

    # Reset the environment to generate the first observation
    observation, info = env.reset()

    # run one episode (max. 1000 steps)
    for _ in range(1000):
        # call the agent to select an action
        action = agent_function(observation)

        # step (transition) through the environment with the action
        observation, reward, terminated, truncated, info = env.step(action)

        if terminated:
            break

    env.close()
    return reward

def run_episodes(agent_function, n=100):
    """Run multiple episodes with the given agent and return the rewards for each episode."""
    return [run_episode_test(agent_function) for _ in range(n)]

rewards = run_episodes(rocket_agent_function)
print(rewards)

print(f"Average reward: {np.average(rewards)}")
print(f"Success rate: {np.sum(np.array(rewards) == 100)}/{len(rewards)}")

[-100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, 100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100]
Average reward: -98.0
Success rate: 1/100


This is not great!


## Implement A Better Reflex-Based Agent

Build a better that uses its right and left thrusters to land the craft (more) safely. Test your agent function using 100 problems.

In [ ]:
# Code goes here
def better_agent_function(observation):
    #calculating the target angle to balance the rocket so that it could land on target
    target_angle = observation[Obs.X.value] * 0.5 + observation[Obs.VX.value] * 0.5

    #calculating the difference between the ideal angle and the actual rocket angle
    angle_error_correction = (target_angle - observation[Obs.ANGLE.value])

    #calculating the ideal thrust
    control_signal = angle_error_correction - observation[Obs.ANGULAR_VELOCITY.value]

    action = Act.NO_OP.value

    #acting based on the control_signal value
    if control_signal > 0.15:
        action = Act.LEFT.value
    elif control_signal < -0.15:
        action = Act.RIGHT.value

    #Decision when rocket angle is not too extreme 
    if abs(observation[Obs.ANGLE.value]) < 0.5:
        if observation[Obs.VY.value] < -0.1:
            action = Act.MAIN.value
    
    #Decision when the left and right leg makes contact on landing surface
    if observation[Obs.LEFT_LEG_CONTACT.value] and observation[Obs.RIGHT_LEG_CONTACT.value]:
        action = Act.NO_OP.value

    return action

better_rewards = run_episodes(better_agent_function)
print(better_rewards)
print(f"Average reward: {np.average(better_rewards)}")
print(f"Success rate: {np.sum(np.array(better_rewards) == 100)}/{len(rewards)}")



[-100, 100, 100, -100, 100, 100, 100, 100, 100, 100, 100, 100, 100, 100, 100, 100, 100, 100, 100, 100, 100, -100, 100, 100, 100, 100, 100, 100, 100, np.float64(-0.2883425350688771), 100, -100, -100, 100, 100, 100, 100, 100, 100, 100, 100, 100, 100, 100, 100, 100, 100, 100, 100, -100, -100, 100, 100, 100, 100, 100, 100, 100, 100, 100, 100, 100, 100, 100, 100, 100, 100, 100, 100, 100, 100, 100, 100, 100, 100, 100, 100, 100, np.float64(-0.006137363436720306), 100, 100, 100, 100, -100, 100, 100, 100, -100, 100, 100, 100, 100, 100, 100, 100, 100, 100, -100, 100, 100]
Average reward: 77.99705520101494
Success rate: 88/100


In [31]:
run_episode(better_agent_function)

Obs: [-0.00427465  1.4047796  -0.43299335 -0.27291524  0.00496005  0.09807926
  0.          0.        ] -> Action: 2
Obs: [-0.0085968   1.3986524  -0.43689576 -0.27234504  0.00958918  0.09259209
  0.          0.        ] -> Action: 2
Obs: [-0.01296129  1.3926814  -0.44094315 -0.26541528  0.01403632  0.08895093
  0.          0.        ] -> Action: 2
Obs: [-0.01723652  1.3874539  -0.4325067  -0.23238607  0.0189717   0.09871672
  0.          0.        ] -> Action: 2
Obs: [-0.02170048  1.383038   -0.45055303 -0.19631709  0.02308816  0.08233689
  0.          0.        ] -> Action: 2
Obs: [-0.02631903  1.3792578  -0.46536374 -0.16806826  0.02655678  0.0693788
  0.          0.        ] -> Action: 2
Obs: [-0.0310359   1.376237   -0.47482842 -0.1343157   0.02967136  0.06229745
  0.          0.        ] -> Action: 2
Obs: [-0.03581304  1.3741384  -0.48071662 -0.09333771  0.03264146  0.05940714
  0.          0.        ] -> Action: 3
Obs: [-0.04051933  1.3714343  -0.4718328  -0.12020251  0.03383179

100